# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [5]:
%pip install ratsnlp

DEPRECATION: pytorch-lightning 1.6.1 has a non-standard dependency specifier torch>=1.8.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


# 구글 드라이브 연동하기

지난 튜토리얼에서 구축한 어휘 집합을 저장해 둔 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [6]:
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)

## GPT 입력값 만들기

GPT 모델 입력값을 만들려면 Byte-level Byte Pair Encoding 어휘집합 구축 결과(`vocab.json`, `merges.txt`)가 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/wordpiece`)에 있어야 합니다. 다음을 수행해 이미 만들어 놓은 BBPE 어휘집합을 포함한 GPT 토크나이저를 `tokenizer_gpt`라는 변수로 선언합니다.

In [7]:
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained("./bbpe")
tokenizer_gpt.pad_token = "[PAD]"

/home/jr/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


예시 문장 세 개를 각각 토큰화해보겠습니다.

In [8]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

토큰화 결과를 확인합니다.

In [9]:
tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĶ',
  'ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]

이번 배치의 크기가 3이라고 가정하고 이번 배치의 입력값을 만들어 보겠습니다.

In [10]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_gpt(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

`batch_inputs`의 내용을 확인해보겠습니다.

In [11]:
batch_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [12]:
batch_inputs['input_ids']

[[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0],
 [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292],
 [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]]

In [13]:
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

## BERT 입력값 만들기

BERT 모델 입력값을 만들려면 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/wordpiece`)에 워드피스 어휘집합 구축 결과(`vocab.txt`)가 있어야 합니다. 다음을 수행해 위에서 이미 만들어 놓은 워드피스 어휘집합을 포함한 BERT 토크나이저를 `tokenizer_bert`라는 변수로 선언합니다.

In [14]:
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(
    "./wordpiece",
    do_lower_case=False,
)

예시 문장 세 개를 각각 토큰화해보겠습니다.

In [15]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

토큰화 결과를 확인합니다.

In [16]:
tokenized_sentences

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'],
 ['흠',
  '.',
  '.',
  '.',
  '포스터',
  '##보고',
  '초딩',
  '##영화',
  '##줄',
  '.',
  '.',
  '.',
  '.',
  '오버',
  '##연기',
  '##조차',
  '가볍',
  '##지',
  '않',
  '##구나'],
 ['별루', '였다', '.', '.']]

이번 배치의 크기가 3이라고 가정하고 이번 배치의 입력값을 만들어 보겠습니다.

In [17]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_bert(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

`batch_inputs`의 내용을 확인해보겠습니다.

In [18]:
batch_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [19]:
batch_inputs['input_ids']

[[2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1304, 16, 3],
 [2, 3274, 9507, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

In [20]:
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]

In [21]:
batch_inputs['token_type_ids']

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]